### Postgres create engine (SQLAlchemy connection)

In [130]:
# manually build path relative to notebook

from pathlib import Path
from dotenv import load_dotenv
import os

# With folder structure:
# epi-dashboard/
# ├─ env/.env
# └─ notebooks/etl_who_data.ipynb

env_path = Path.cwd().parent / "env" / ".env"  # go up one folder from /notebooks
load_dotenv(dotenv_path=env_path) # tell it where env file is

# Postgres credentials should now be available via os.getenv, Default is current working directory or parent folder

True

In [131]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text

pg_url = URL.create(
    "postgresql+psycopg",
    username=os.getenv("PGUSER"),
    password=os.getenv("PGPASSWORD"),
    host=os.getenv("PGHOST", "localhost"),
    port=int(os.getenv("PGPORT", 5432)),
    database=os.getenv("PGDATABASE"),
)

engine = create_engine(pg_url, pool_pre_ping=True)


In [132]:
# Alternative syntax

# engine = create_engine(
#     f"postgresql+psycopg2://{os.environ['PGUSER']}:{os.environ['PGPASSWORD']}@"
#     f"{os.environ['PGHOST']}:{os.environ['PGPORT']}/{os.environ['PGDATABASE']}"
# )


### Test connection

In [133]:
try:
    with engine.begin() as conn:
        version = conn.execute(text("SELECT version();")).scalar()
        print("Connected to:", version)
        conn.execute(text("CREATE SCHEMA IF NOT EXISTS gho;"))
except Exception as e:
    print("Connection failed:", e)


Connected to: PostgreSQL 17.6 on x86_64-windows, compiled by msvc-19.44.35213, 64-bit


In [134]:
# error is from SQLAlchemy 2.0: plain SQL strings aren’t implicitly executable anymore:
# with engine.connect() as conn:
#     print(conn.execute("SELECT version();").fetchone())

# So Wrap the SQL in text() 

#### Just life expectancy at birth to start with (WHOSIS_000001)

You’ll see fields like IndicatorCode, IndicatorName, SpatialDimType, SpatialDim (ISO3), ParentLocation, TimeDim (year), Dim1 (sex code), NumericValue, Low, High, TimeDimensionBegin, TimeDimensionEnd, etc. (Field set is documented and used consistently across indicators.)

In [77]:
import requests, pandas as pd, time

BASE = "https://ghoapi.azureedge.net/api/WHOSIS_000001"

# OPTIONAL: pre-filter at the source to shrink what you download
# e.g., both sexes only:
# BASE += "?$filter=Dim1 eq 'BTSX'"

frames = []
url = BASE  # start with the base (or filtered) URL

while True:
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    js = r.json()
    if 'value' in js and js['value']:
        frames.append(pd.json_normalize(js['value']))
    # follow server-driven paging
    url = js.get('@odata.nextLink')
    if not url:
        break
    time.sleep(0.1)  # polite pause

raw = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
raw.shape


(12936, 25)

In [78]:
#Check what API returned to environment 
print(sorted(raw.columns.tolist()))

['Comments', 'DataSourceDim', 'DataSourceDimType', 'Date', 'Dim1', 'Dim1Type', 'Dim2', 'Dim2Type', 'Dim3', 'Dim3Type', 'High', 'Id', 'IndicatorCode', 'Low', 'NumericValue', 'ParentLocation', 'ParentLocationCode', 'SpatialDim', 'SpatialDimType', 'TimeDim', 'TimeDimType', 'TimeDimensionBegin', 'TimeDimensionEnd', 'TimeDimensionValue', 'Value']


#### Tidying dataframe

In [79]:
import pandas as pd
from datetime import date

# 1) Inspect what actually came back (for debugging only)
print(sorted(raw.columns.tolist())[:40])  # peek at the first ~40 columns

# 2) Prefer country rows if possible; otherwise fall back to ISO3-looking codes
if 'SpatialDimType' in raw.columns:
    countries = raw[raw['SpatialDimType'].eq('COUNTRY')].copy()
else:
    # fallback: rows with a 3-letter all-caps SpatialDim (looks like an ISO3 code)
    countries = raw[raw['SpatialDim'].astype(str).str.fullmatch(r'[A-Z]{3}', na=False)].copy()

# 3) Robust mapping with fallbacks (only keep/rename columns that exist; fill missing)
# If absent, target column created with NA
source_to_target = {
    'IndicatorCode': 'indicator_code',
    'IndicatorName': 'indicator_name',          # may be absent
    'SpatialDimType': 'spatial_dim_type',
    'SpatialDim': 'spatial_dim',
    'ParentLocation': 'parent_location',        # may be absent
    'TimeDim': 'time_dim',
    'Dim1': 'dim1',
    'NumericValue': 'numeric_value',
    'Low': 'low',
    'High': 'high',
    'Value': 'value',                            # may be absent
    'TimeDimensionBegin': 'time_dimension_begin',
    'TimeDimensionEnd': 'time_dimension_end',
}

# Build the target DataFrame column-by-column
df_parts = {}
for src, tgt in source_to_target.items():
    if src in countries.columns:
        df_parts[tgt] = countries[src]
    else:
        # sensible default NA types
        if tgt in {'time_dim'}:
            df_parts[tgt] = pd.Series(pd.NA, index=countries.index, dtype='Int64')
        elif tgt in {'numeric_value','low','high'}:
            df_parts[tgt] = pd.Series(pd.NA, index=countries.index, dtype='float')
        elif tgt in {'time_dimension_begin','time_dimension_end'}:
            df_parts[tgt] = pd.Series(pd.NaT, index=countries.index, dtype='datetime64[ns]')
        else:
            df_parts[tgt] = pd.Series(pd.NA, index=countries.index, dtype='object')

df = pd.DataFrame(df_parts)

# 4) Type casting & cleaning
if 'time_dim' in df:
    df['time_dim'] = df['time_dim'].astype('Int64')

for c in ['numeric_value', 'low', 'high']:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors='coerce')

for c in ['time_dimension_begin', 'time_dimension_end']:
    if c in df:
        df[c] = pd.to_datetime(df[c], errors='coerce').dt.date

# 5) Optional: if indicator_name is missing, derive a friendly one from indicator_code
if df['indicator_code'].notna().any() and df['indicator_name'].isna().all():
    df['indicator_name'] = 'Life expectancy at birth (WHOSIS_000001)'

df.head(3)


['Comments', 'DataSourceDim', 'DataSourceDimType', 'Date', 'Dim1', 'Dim1Type', 'Dim2', 'Dim2Type', 'Dim3', 'Dim3Type', 'High', 'Id', 'IndicatorCode', 'Low', 'NumericValue', 'ParentLocation', 'ParentLocationCode', 'SpatialDim', 'SpatialDimType', 'TimeDim', 'TimeDimType', 'TimeDimensionBegin', 'TimeDimensionEnd', 'TimeDimensionValue', 'Value']


,indicator_code,indicator_name,spatial_dim_type,spatial_dim,parent_location,time_dim,dim1,numeric_value,low,high,value,time_dimension_begin,time_dimension_end
0,WHOSIS_000001,Life expectancy at birth (WHOSIS_000001),COUNTRY,SOM,Eastern Mediterranean,2008,SEX_MLE,48.037541,46.716784,49.628457,48.0 [46.7-49.6],2008-01-01,2008-12-31
1,WHOSIS_000001,Life expectancy at birth (WHOSIS_000001),COUNTRY,BTN,South-East Asia,2002,SEX_BTSX,67.845665,67.083124,68.558745,67.8 [67.1-68.6],2002-01-01,2002-12-31
2,WHOSIS_000001,Life expectancy at birth (WHOSIS_000001),COUNTRY,BHR,Eastern Mediterranean,2011,SEX_FMLE,75.205359,75.050478,75.385175,75.2 [75.1-75.4],2011-01-01,2011-12-31


#### Push into postgres

In [80]:
df.to_sql("life_expectancy", engine, schema="gho", if_exists="append", index=False)


-1

In [81]:
# Optional: analyze for better plans
with engine.begin() as conn:
    conn.execute(text("ANALYZE gho.life_expectancy;"))

#### Query with SQL

In [138]:
import pandas as pd

query = "SELECT * FROM gho.life_expectancy LIMIT 10;"  # adjust LIMIT as needed
df = pd.read_sql(query, engine)
df.head()

,indicator_code,spatial_dim_type,spatial_dim,parent_location,time_dim,dim1,numeric_value,low,high,value,time_dimension_begin,time_dimension_end,indicator_name
0,WHOSIS_000001,COUNTRY,SOM,Eastern Mediterranean,2008,SEX_MLE,48.04,46.72,49.63,48.0 [46.7-49.6],2008-01-01,2008-12-31,Life expectancy at birth (WHOSIS_000001)
1,WHOSIS_000001,COUNTRY,BTN,South-East Asia,2002,SEX_BTSX,67.85,67.08,68.56,67.8 [67.1-68.6],2002-01-01,2002-12-31,Life expectancy at birth (WHOSIS_000001)
2,WHOSIS_000001,COUNTRY,BHR,Eastern Mediterranean,2011,SEX_FMLE,75.21,75.05,75.39,75.2 [75.1-75.4],2011-01-01,2011-12-31,Life expectancy at birth (WHOSIS_000001)
3,WHOSIS_000001,COUNTRY,SAU,Eastern Mediterranean,2005,SEX_FMLE,73.12,72.81,73.48,73.1 [72.8-73.5],2005-01-01,2005-12-31,Life expectancy at birth (WHOSIS_000001)
4,WHOSIS_000001,COUNTRY,CYP,Europe,2003,SEX_MLE,76.86,76.67,77.16,76.9 [76.7-77.2],2003-01-01,2003-12-31,Life expectancy at birth (WHOSIS_000001)


In [140]:
pd.read_sql("SELECT column_name, data_type FROM information_schema.columns WHERE table_schema='gho' AND table_name='life_expectancy';", engine)


,column_name,data_type
0,indicator_code,text
1,spatial_dim_type,text
2,spatial_dim,character
3,parent_location,text
4,time_dim,integer
5,dim1,text
6,numeric_value,numeric
7,low,numeric
8,high,numeric
9,value,text


In [141]:
q = """
SELECT iso3, time_begin, value
FROM fact_observation
WHERE indicator_code='WHOSIS_000001'
ORDER BY time_begin;
"""
lifeexp = pd.read_sql(q, engine)
lifeexp.head()


,iso3,time_begin,value


other

### Ingest WHO data into postgres (requests + pagination + pandas → Postgres)

Notes:
SQLAlchemy2.0 needs an explicit connection created (or a session or ORM) to execute on

SQL always wrapped in text()

with ENGINE.begin() so you get a proper transaction and auto-commit

Pass parameters as a dict instead of positional %s

Replace every ENGINE.execute(...) with either:

with ENGINE.begin() as conn: conn.execute(text(...), params)

Or, if you’ll be doing a lot of inserts/updates, create a Session from sqlalchemy.orm and use that.

In [27]:
# Fetching indicator names from API before upserting 
# Pull the Indicator catalog once, map IndicatorCode → IndicatorName, then use it in the upsert.

import requests
import pandas as pd
from sqlalchemy import text

BASE = "https://ghoapi.azureedge.net/api"

def odata_pages(url: str):
    while url:
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        data = r.json()
        yield data.get("value", [])
        url = data.get("@odata.nextLink")

def build_indicator_name_map() -> dict[str, str]:
    url = f"{BASE}/Indicator?$select=IndicatorCode,IndicatorName"
    names = {}
    for page in odata_pages(url):
        for row in page:
            code = row.get("IndicatorCode")
            name = row.get("IndicatorName")
            if code and name:
                names[code] = name
    return names

INDICATOR_NAMES = build_indicator_name_map()   # call once at start

def upsert_dim_indicator(ind_code: str, topic: str | None = None, unit: str | None = None):
    name = INDICATOR_NAMES.get(ind_code, ind_code)  # fall back to code if unknown
    sql = text("""
        INSERT INTO dim_indicator (indicator_code, indicator_name, topic, unit)
        VALUES (:code, :name, :topic, :unit)
        ON CONFLICT (indicator_code) DO UPDATE
        SET indicator_name = EXCLUDED.indicator_name,
            topic          = COALESCE(EXCLUDED.topic, dim_indicator.topic),
            unit           = COALESCE(EXCLUDED.unit,  dim_indicator.unit)
    """)
    with ENGINE.begin() as conn:
        conn.execute(sql, {"code": ind_code, "name": name, "topic": topic, "unit": unit})


### COVID weekly loader

In [ ]:
from sqlalchemy import text

with ENGINE.begin() as conn:
    conn.execute(text("""
        INSERT INTO dim_location (iso3, country, who_region)
        VALUES (:iso3, :country, :who_region)
        ON CONFLICT (iso3) DO NOTHING
    """), [{"iso3":"GBR","country":"United Kingdom","who_region":"EURO"}])
